# The Julia Programming language

Some short examples

In [2]:
#Simple math
x = 2
y = 3
x + y

5

In [3]:
# Vector math
x = [1,2,3,4]
y = [2.5,3.5,1.0,2.9]
x + y

4-element Array{Float64,1}:
 3.5
 5.5
 4.0
 6.9

In [4]:
# Functions look like math
f(x) = 2x^2 + sin(x)
f(5)

49.04107572533686

In [5]:
# Apply a function element-wise
f.(x)

4-element Array{Float64,1}:
  2.8414709848078967
  8.909297426825681 
 18.14112000805987  
 31.243197504692073 

In [7]:
# Always be explicit
sin.([1.0,2.0,3.0])

3-element Array{Float64,1}:
 0.8414709848078965
 0.9092974268256817
 0.1411200080598672

In [8]:
# Matrix math
m = rand(4,3)
n = ones(3,5)
m*n

4×5 Array{Float64,2}:
 1.94828  1.94828  1.94828  1.94828  1.94828
 1.84022  1.84022  1.84022  1.84022  1.84022
 1.76882  1.76882  1.76882  1.76882  1.76882
 1.13088  1.13088  1.13088  1.13088  1.13088

In [9]:
m = rand(3,3)
n = ones(3,3)
m.*n == m*n

false

In [14]:
m .== n

3×3 BitArray{2}:
 false  false  false
 false  false  false
 false  false  false

### So what is so special about Julia?

* fast, generic and dynamic at the same time

### Difference between static and dynamic typing 

C:

````C
double f(double x, double y) {
    double r;
    r = x + y;
    r = r + x;
    return r;
}
````

* variable have types
* within `f` x and y are always double floating point numbers
* all operations are predictable for the compiler
* the return type is known
* code is fast but not generic

R:

````R
f <- function(x, y) {
    r <- x + y
    r <- r + x
    return r
}
````

* values have types
* f is generic, because the operations executed depend on the inputs
* x and y might be numeric vectors, raster objects, DataFrames...
* everything is possible as long as `+` is defined for the type
* code is usually slower because the compiler has to check line by line which operation to execute
* return type is unknown

### Julia

In [15]:
function f(x,y)
    r = x + y
    r = r + x
    r
end

f (generic function with 2 methods)

In [17]:
@code_lowered f(x,y)

CodeInfo(
│2 1 ─     r = x + y
│3 │       r = r + x
│4 └──     return r
)

In [23]:
@code_typed f(2.0,5.0)

CodeInfo(
│╻ +2 1 ─ %1 = (Base.add_float)(x, y)::Float64
│╻ +3 │   %2 = (Base.add_float)(%1, x)::Float64
│ 4 └──      return %2
) => Float64

In [19]:
@code_llvm f(2,3)


; Function f
; Location: In[15]:2
define i64 @julia_f_36024(i64, i64) {
top:
  %factor = shl i64 %0, 1
; Location: In[15]:3
; Function +; {
; Location: int.jl:53
  %2 = add i64 %factor, %1
;}
; Location: In[15]:4
  ret i64 %2
}


In [20]:
@code_native f(2,3)

	.text
; Function f {
; Location: In[15]:3
; Function +; {
; Location: In[15]:2
	leaq	(%rsi,%rdi,2), %rax
;}
; Location: In[15]:4
	retq
	nopw	%cs:(%rax,%rax)
;}


### Implications for Julia code

* type stability (sqrt)
* eval only in global scope
* use of *function barriers* to separate type-unstable from type-stable parts

In [27]:
sqrt(Complex(2.0))

1.4142135623730951 + 0.0im

In [28]:
x = :(a = 5)

:(a = 5)

In [29]:
eval(x)

5

In [30]:
a

5

In [33]:
function doeval()
    x = :(b = 5)
    eval(x)
    b
end

doeval (generic function with 1 method)

In [36]:
@code_warntype doeval()

Body::Any
│2 1 ─ %1 = $(Expr(:copyast, :($(QuoteNode(:(b = 5))))))::Core.Compiler.Const(:(b = 5), false)
│3 │        invoke Main.eval(%1::Expr)
│4 └──      return Main.b


In [35]:
b

5